In [117]:
import io
from PIL import Image
import json
import cv2
import numpy as np
import requests
import requests

class Receipt:
    def __init__(self,image):
        self.data = ''
        self.im_file = image
        self.words = ''
        #Send image to memory
        self.im = Image.open(self.im_file)

#Request function 

    def ocr_space_file(self,filename, overlay=False, api_key='helloworld', language='eng'):
        """ OCR.space API request with local file.
            Python3.5 - not tested on 2.7
        :param filename: Your file path & name.
        :param overlay: Is OCR.space overlay required in your response.
                        Defaults to False.
        :param api_key: OCR.space API key.
                        Defaults to 'helloworld'.
        :param language: Language code to be used in OCR.
                        List of available language codes can be found on https://ocr.space/OCRAPI
                        Defaults to 'en'.
        :return: Result in JSON format.
        """

        payload = {'isOverlayRequired': overlay,
                'apikey': api_key,
                'language': language,
                }
        with open(filename, 'rb') as f:
            r = requests.post('https://api.ocr.space/parse/image',
                            files={filename: f},
                            data=payload,
                            )
        return r.content.decode()
    
    def make_request(self):
        raw_data  = self.ocr_space_file(self.im_file,overlay=True,api_key='K89127144588957',language='eng')
        self.data = json.loads(raw_data)

    #Print data

    def print_data(self):
        print(self.data)

    #Print Keys

    def print_keys(self):
        print(self.data.keys())

    #Text overlay (get only the text and coordinates, but no the final details)
    
    def get_text_overlay(self):
        text_overlay = self.data.get('ParsedResults')[0].get('TextOverlay')
        return text_overlay

    #Returns only the strings the API found

    def get_parsed_text(self):
        parsed_text = self.data.get('ParsedResults')[0].get('ParsedText')
        return parsed_text
    
    #Get lines and coordinates based on an index from data

    def get_line_and_coordinate(self,i):
        lines_coordinate = self.get_text_overlay().get('Lines')
        return lines_coordinate
    
    #Get only the string from line based on an index

    def get_line(self,i):
        line = self.get_line_and_coordinate(i)[i].get('LineText')
        return line
    
    #Set words

    def set_words(self,i):
        self.words = self.get_words(i)

    #Get words (word and coordinate)

    def get_words(self,i):
        word = self.get_line_and_coordinate(i)[i].get('Words')
        return word

                #Get coordinates main funcion
                #Since the values are given as a distance from te edge of the picture to the word, the coordinates were calculated with max point as min and min as max)

    def find_highest_point(self):
        highest = 0
        for words in self.words:
            highest_point = words['Height']
            if highest_point > highest:
                highest = highest_point
        return highest
    def find_max_point(self):
        im = cv2.imread(self.im_file)
        max = im.shape[0]
        for words in self.words:
            max_point = words['Top']
            if max_point < max:
                max=max_point
        return max
    def find_min_point(self):
        highest_point_word = self.find_highest_point()
        min = 0
        for words in self.words:
            min_point = words['Top']
            if min_point > min:
                min=min_point
        return min+highest_point_word

        #This is the function we will use to get the coordinates of the text in the image
    def get_coordinates(self):
        max_word_point = self.find_max_point()
        min_word_point = self.find_min_point()
        first_word = self.words[0]
        last_word = self.words[-1]
        x1,y1 = first_word['Left'], max_word_point
        x2,y2 = last_word['Left'] + last_word['Width'],max_word_point
        x3,y3 = first_word['Left'], min_word_point
        x4,y4 = last_word['Left'] + last_word['Width'], min_word_point
        return [[x1,y1],[x2,y2],[x3,y3],[x4,y4]]
    
#Draw bounding boxed based on the words set. We will probably need a for loop that iterates n times to draw n boxes   

    def draw_bounding_box(self):
        coordinates = self.get_coordinates()
        x1_left = coordinates[0][0]
        y1_left = coordinates[0][1]
        x4_right = coordinates[-1][0]
        y4_right = coordinates[-1][1]
        start_point = (int(x1_left),int(y1_left))
        end_point = (int(x4_right),int(y4_right))
        self.image = cv2.imread(self.im_file)
        cv2.rectangle(self.image,start_point,end_point,color=(0,255,0),thickness=2)
        cv2.imwrite('Receipts/second_with_bounding_forth.jpg',self.image)
    




In [118]:
receipt = Receipt('Receipts/second.jpg')
receipt.make_request()



In [119]:
receipt.set_words(4)
receipt.get_line(4)

'Jalan Kasuarina 1,'

In [120]:
receipt.get_coordinates()

[[233.0, 279.0], [459.0, 279.0], [233.0, 306.0], [459.0, 306.0]]

In [121]:

receipt.draw_bounding_box()

In [122]:
#Get coordinates and string value
text_values = receipt.get_parsed_text().split('\r\n')
try:
    for i,values in enumerate(text_values):
        receipt.set_words(i)
        print(receipt.get_coordinates(),receipt.get_line(i))
        receipt.draw_bounding_box()
except:
    None


[[198.0, 131.0], [483.0, 131.0], [198.0, 178.0], [483.0, 178.0]] Sarang Hae Yo
[[184.0, 182.0], [500.0, 182.0], [184.0, 211.0], [500.0, 211.0]] TRENDYMAX (M) SON. BHD.
[[160.0, 211.0], [529.0, 211.0], [160.0, 246.0], [529.0, 246.0]] Company Reg. No. : (583246•A).
[[104.0, 244.0], [586.0, 244.0], [104.0, 279.0], [586.0, 279.0]] PS, Block C, GM Klang Wholesale City,
[[233.0, 279.0], [459.0, 279.0], [233.0, 306.0], [459.0, 306.0]] Jalan Kasuarina 1,
[[172.0, 312.0], [521.0, 312.0], [172.0, 343.0], [521.0, 343.0]] 41200 Klang, Selangor D.E.
[[158.0, 348.0], [525.0, 348.0], [158.0, 384.0], [525.0, 384.0]] Whatsapp : +6012-5060668
[[157.0, 392.0], [517.0, 392.0], [157.0, 426.0], [517.0, 426.0]] www.saranghaeyo.com
[[196.0, 436.0], [500.0, 436.0], [196.0, 470.0], [500.0, 470.0]] GST Reg : 000384098304
[[255.0, 513.0], [444.0, 513.0], [255.0, 538.0], [444.0, 538.0]] TAX INVOICE
[[33.0, 559.0], [253.0, 559.0], [33.0, 582.0], [253.0, 582.0]] CB#: GM3-46792
[[33.0, 602.0], [360.0, 602.0], [33.0, 

In [124]:
receipt.set_words(70)
receipt.draw_bounding_box()